In [1]:
import os
import tensorflow as tf
os.chdir('../')

In [2]:
from firstClassifier.constants import *
from firstClassifier.utils.common import read_yaml, create_directories

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
  root_dir: Path
  trained_model_path: Path
  updated_base_model_path: Path
  training_data: Path
  params_epochs: int
  params_batch_size: int
  params_is_augmentation: bool
  params_image_size: list
  

In [4]:
class ConfigurationManager:

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
             ):
        """_summary_: Creates artifact directories taken from config.yaml file
        """
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        ##Create directories
        create_directories([self.config.artifacts_root])

    def get_training_config(self)->TrainingConfig:
        training= self.config.training
        training_data= os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config= TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path= self.config.prepare_base_model.updated_base_model_path,
            training_data= Path(training_data),
            params_epochs= self.params.EPOCHS,
            params_batch_size= self.params.BATCH_SIZE,
            params_is_augmentation= self.params.AUGMENTATION,
            params_image_size= self.params.IMAGE_SIZE
        )

        return training_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
class Training:
    def __init__(self, config: TrainingConfig):
        self.config= config

    def get_base_model(self):
        self.model= tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        ##Augmentation
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split= 0.20

        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size,
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle= False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
            
        else:
            train_datagenerator= valid_datagenerator

        self.train_generator= train_datagenerator.flow_from_dataframe(
            directory=self.config.training_data,
            subset="training",
            shuffle= True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)

        
    def train(self, callback_list):

        self.steps_per_epoch= self.train_generator.samples// self.train_generator.batch_size
        self.validation_steps= self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs= self.config.params_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_steps= self.validation_steps,
            validation_data= self.valid_generator,
            callbacks= callback_list
        )

        self.save_model(
            path= self.config.trained_model_path,
            model= self.model
        )